# 라이브러리 불러오기

In [2]:
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings

warnings.filterwarnings('ignore')

# 랜덤 시드 설정

In [3]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [4]:
RAW_BASE_PATH = os.path.join("..", "..", "data", "raw")
PROCESSED_BASE_PATH = os.path.join("..", "..", "data", "processed")
# train_data = pd.read_csv(os.path.join(BASE_PATH, 'train.csv'))
# test_data = pd.read_csv(os.path.join(BASE_PATH, 'test.csv'))
raw = pd.read_csv(os.path.join(PROCESSED_BASE_PATH, 'V1.csv'))
sample_submission = pd.read_csv(os.path.join(RAW_BASE_PATH, 'sample_submission.csv'))
print(raw.columns)
raw.head()

Index(['area_m2', 'contract_type', 'floor', 'built_year', 'latitude',
       'longitude', 'age', 'deposit', 'contract_ymd', 'recent_deposit',
       ...
       'sb_610', 'sb_436', 'sb_658', 'sb_37', 'sb_53', 'sb_341', 'sb_69',
       'sb_645', 'sb_614', 'sb_94'],
      dtype='object', length=563)


,area_m2,contract_type,floor,built_year,latitude,longitude,age,deposit,contract_ymd,recent_deposit,...,sb_610,sb_436,sb_658,sb_37,sb_53,sb_341,sb_69,sb_645,sb_614,sb_94
0,72.320,2,16,2016,36.977063,126.928605,3,17000.0,2019-04-01,-999.0,...,0,0,0,0,0,0,0,0,0,0
1,22.270,2,11,2013,36.989441,126.842601,6,4000.0,2019-04-01,-999.0,...,0,0,0,0,0,0,0,0,0,0
2,59.940,2,9,1996,37.126312,127.070305,23,9000.0,2019-04-01,-999.0,...,0,0,0,0,0,0,0,0,0,0
3,59.760,2,1,1992,36.961599,126.918827,27,3000.0,2019-04-01,-999.0,...,0,0,0,0,0,0,0,0,0,0
4,58.764,2,12,2005,37.589086,127.206811,14,18000.0,2019-04-01,-999.0,...,0,0,0,0,0,0,0,0,0,0


## APT IDX 별로 Subway OneHot Coding Feature 추가


In [48]:
from typing import List

sb_one_hot_columns = [c if c.startswith("sb_") else "" for c in raw.columns]
sb_one_hot_columns: List[str] = np.unique(sb_one_hot_columns).tolist()
sb_one_hot_columns.remove("")
sb_one_hot_columns

['sb_0',
 'sb_1',
 'sb_10',
 'sb_100',
 'sb_101',
 'sb_102',
 'sb_103',
 'sb_104',
 'sb_105',
 'sb_106',
 'sb_107',
 'sb_108',
 'sb_109',
 'sb_11',
 'sb_110',
 'sb_111',
 'sb_112',
 'sb_113',
 'sb_114',
 'sb_115',
 'sb_117',
 'sb_118',
 'sb_119',
 'sb_12',
 'sb_120',
 'sb_121',
 'sb_122',
 'sb_123',
 'sb_124',
 'sb_125',
 'sb_126',
 'sb_127',
 'sb_128',
 'sb_129',
 'sb_13',
 'sb_130',
 'sb_131',
 'sb_132',
 'sb_133',
 'sb_134',
 'sb_135',
 'sb_136',
 'sb_137',
 'sb_138',
 'sb_139',
 'sb_14',
 'sb_141',
 'sb_142',
 'sb_143',
 'sb_144',
 'sb_146',
 'sb_147',
 'sb_148',
 'sb_149',
 'sb_15',
 'sb_150',
 'sb_151',
 'sb_152',
 'sb_153',
 'sb_154',
 'sb_155',
 'sb_156',
 'sb_157',
 'sb_158',
 'sb_159',
 'sb_160',
 'sb_161',
 'sb_162',
 'sb_163',
 'sb_165',
 'sb_166',
 'sb_167',
 'sb_170',
 'sb_171',
 'sb_172',
 'sb_173',
 'sb_174',
 'sb_175',
 'sb_176',
 'sb_177',
 'sb_178',
 'sb_179',
 'sb_180',
 'sb_181',
 'sb_182',
 'sb_183',
 'sb_184',
 'sb_185',
 'sb_186',
 'sb_187',
 'sb_188',
 'sb_189'

In [56]:
select_col = ["apt_idx", "latitude", "longitude"]
select_col.extend(sb_one_hot_columns)
raw = raw[select_col]


MemoryError: Unable to allocate 7.92 GiB for an array with shape (545, 1951400) and data type int64

In [76]:
from tqdm import tqdm

apt_idx = pd.DataFrame(raw["apt_idx"].drop_duplicates())
apt_idx.set_index("apt_idx", inplace=True)
for c in tqdm(raw.columns):
    if c == "apt_idx":
        continue
    right_df = raw[["apt_idx", c]]
    right_df.drop_duplicates(inplace=True)
    right_df.set_index("apt_idx", inplace=True)
    apt_idx = apt_idx.join(right_df, on="apt_idx", how="left")
apt_idx.sort_index(inplace=True)
apt_idx

100%|██████████| 547/547 [01:01<00:00,  8.89it/s]


,latitude,longitude,sb_0,sb_1,sb_10,sb_100,sb_101,sb_102,sb_103,sb_104,...,sb_9,sb_90,sb_91,sb_92,sb_94,sb_95,sb_96,sb_97,sb_98,sb_99
apt_idx,,,,,,,,,,,,,,,,,,,,,
15,36.977063,126.928605,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,36.989441,126.842601,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,37.126312,127.070305,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
64,36.961599,126.918827,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
67,37.589086,127.206811,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18529,37.181505,127.111240,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18628,37.485175,126.687350,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18570,37.551718,127.022924,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:
apt_idx.to_csv(os.path.join(PROCESSED_BASE_PATH, 'apt_idx_subway_onehot.csv'), index=False)
del apt_idx, select_col, sb_one_hot_columns

## 기타 Feature EDA

In [8]:
if "recent_price" not in raw.columns:
    raw["recent_m2_price"] = raw["recent_deposit"] / raw["area_m2"]


In [14]:
from tqdm import tqdm

corr_raw = raw.drop(
    columns=["contract_ymd", "nearest_subway_distance", "nearest_subway_idx", "list_subway_idx_within_1km", "latitude",
             "longitude"])

corr_raw["recent_m2_price"] = np.log(corr_raw["recent_m2_price"])
corr_raw["area_m2_price"] = np.log(corr_raw["area_m2_price"])
corr_raw["is_new"] = 0
corr_raw.loc[corr_raw["contract_type"] == 0, "is_new"] = 1
corr_raw["is_renew"] = 0
corr_raw.loc[corr_raw["contract_type"] == 1, "is_renew"] = 1
corr_result = None
target = "area_m2_price"
for c in tqdm(corr_raw.columns):
    if c == target:
        continue
    corr_row = corr_raw[[target, c]].corr()[1:]
    if corr_result is None:
        corr_result = corr_row
        continue

    corr_result = pd.concat([corr_result, corr_row], axis=0)[target]

# corr_result.to_csv(f"corr_{target}_subway_one_hot.csv")
corr_result
# corr_raw

100%|██████████| 560/560 [00:21<00:00, 26.58it/s]


area_m2           -0.005295
contract_type     -0.131637
floor              0.107049
built_year         0.165861
age               -0.152583
                     ...   
sb_614            -0.010081
sb_94             -0.005212
recent_m2_price    0.863022
is_new             0.099043
is_renew           0.065471
Name: area_m2_price, Length: 559, dtype: float64

In [19]:
corr_result.sort_values(ascending=False).to_csv("log_area_m2_price.csv", index=False)

## 상관관계 계수 10% 이상인 Subway 위치 표시

In [36]:

import folium

'''
상관계수 높은 애들(0.1)만 뽑아내기.
'''
sub_list = []
for idx in corr_result[corr_result > 0.1].index:
    if str(idx).startswith("sb_"):
        sub_list.append(int(str(idx).replace("sb_", "")))

# Subway 정보에서 상관관계 높은 애들만 추출
subway_df = pd.read_csv(os.path.join(RAW_BASE_PATH, "subwayInfo.csv"))
subway_df = subway_df.loc[sub_list]
print(subway_df.shape)
print(subway_df)

lat_mean = subway_df['latitude'].mean()
lon_mean = subway_df['longitude'].mean()
map = folium.Map(location=[lat_mean, lon_mean], zoom_start=14)
for _, row in subway_df.iterrows():
    m = folium.Marker([row["latitude"], row["longitude"]])
    m.add_to(map)
map

(12, 2)
      latitude   longitude
162  37.512759  127.011220
335  37.508178  127.011727
163  37.504810  127.004943
536  37.496237  127.052873
539  37.489116  127.066140
540  37.491373  127.072720
152  37.494612  127.063642
538  37.486839  127.058856
101  37.511687  127.086162
150  37.486947  127.046769
428  37.504206  127.015259
151  37.490858  127.055381


In [37]:
'''
상관계수 높은 애들(0.09)만 뽑아내기.
'''
sub_list = []
for idx in corr_result[corr_result > 0.09].index:
    if str(idx).startswith("sb_"):
        sub_list.append(int(str(idx).replace("sb_", "")))

# Subway 정보에서 상관관계 높은 애들만 추출
subway_df = pd.read_csv(os.path.join(RAW_BASE_PATH, "subwayInfo.csv"))
subway_df = subway_df.loc[sub_list]
print(subway_df.shape)
print(subway_df)

lat_mean = subway_df['latitude'].mean()
lon_mean = subway_df['longitude'].mean()
map = folium.Map(location=[lat_mean, lon_mean], zoom_start=14)
for _, row in subway_df.iterrows():
    m = folium.Marker([row["latitude"], row["longitude"]])
    m.add_to(map)
map

(19, 2)
      latitude   longitude
249  37.553736  126.956820
162  37.512759  127.011220
335  37.508178  127.011727
163  37.504810  127.004943
107  37.493415  127.014080
104  37.504503  127.049008
536  37.496237  127.052873
539  37.489116  127.066140
540  37.491373  127.072720
153  37.496663  127.070594
152  37.494612  127.063642
538  37.486839  127.058856
367  37.519365  127.053350
101  37.511687  127.086162
100  37.513950  127.102234
150  37.486947  127.046769
428  37.504206  127.015259
419  37.504598  127.025060
151  37.490858  127.055381


## 모델링

In [39]:
raw = raw[
    corr_result[abs(corr_result) > 0.08].index.append(
        "contract_ymd")]  #.to_csv("V1_apt_idx_recent_deposit_sb_one_hot_corr_ap_8.csv", index=False)

In [41]:

train_data = raw.loc[raw["deposit"] != -999]
test_data = raw.loc[raw["deposit"] == -999]

In [42]:
print("train, test_data, sample_submission shape : ", train_data.shape, test_data.shape, sample_submission.shape)
print("train data 상단 5개 정보 확인 : ", train_data.head())
print("test data 상단 5개 정보 확인 : ", test_data.head())
print("sample_submission 상단 5개 정보 확인 : ", sample_submission.head())
print('train data 변수 요약 정보 확인 : ', train_data.describe())
print("test data 변수 요약 정보 확인 : ", test_data.describe())



train, test_data, sample_submission shape :  (1801228, 45) (150172, 45) (150172, 2)
train data 상단 5개 정보 확인 :     floor  built_year  deposit  recent_deposit  apt_idx  num_subway_within_1km  \
0     16        2016  17000.0          -999.0       15                      0   
1     11        2013   4000.0          -999.0       36                      0   
2      9        1996   9000.0          -999.0       53                      0   
3      1        1992   3000.0          -999.0       64                      0   
4     12        2005  18000.0          -999.0       67                      1   

   category_interchange_within_1km  num_subway_within_500m  \
0                                0                       0   
1                                0                       0   
2                                0                       0   
3                                0                       0   
4                                1                       1   

   category_interchange_within

In [44]:
# holdout_start = "2023-07-01"
# holdout_end = "2023-12-01"
# holdout_data = train_data[
#     (train_data['contract_ymd'] >= holdout_start) & (train_data['contract_ymd'] <= holdout_end)]
# train_data = train_data[
#     ~((train_data['contract_ymd'] >= holdout_start) & (train_data['contract_ymd'] <= holdout_end))]
# # 학습 데이터와 정답 데이터 분리
target = "area_m2_price"
train_data
X_train = train_data.drop(columns=[target, ])
y_train = train_data[target]
X_holdout = train_data.drop(columns=[target])
y_holdout = train_data[target]
X_test = test_data.copy()
# LightGBM 모델 훈련
lgb_model = lgb.LGBMRegressor(random_state=RANDOM_SEED)
lgb_model.fit(X_train, y_train)
# Holdout 데이터셋에 대한 성능 확인
lgb_holdout_pred = lgb_model.predict(X_holdout)
lgb_holdout_mae = mean_absolute_error(y_holdout, lgb_holdout_pred)
print("Holdout 데이터셋 성능:")
print(f"LightGBM MAE: {lgb_holdout_mae:.2f}")
# Sample Submission 제출하기
lgb_test_pred = lgb_model.predict(X_test)
sample_submission['deposit'] = lgb_test_pred
sample_submission.to_csv('output.csv', index=False, encoding='utf-8-sig')

KeyError: "['area_price'] not found in axis"